### Bioinformatics 2 - Lecture 5
### Networks

Some code to let you practice working with NetworkX which has vast functionality. The main documentation can be found [here](https://networkx.github.io/documentation/stable/).

There have been some issues with compatibility with various versions of NetworkX and Matplotlib, this Notebook is written for networkx-2.2 and matplotlib-1.4.3

#### ian.simpson@ed.ac.uk
#### Wednesday 13th February 2019

In [ ]:
!pip install networkx==2.2
!pip install seaborn

import warnings
import networkx as nx
import matplotlib.pyplot as plt
import matplotlib.cbook as cb
import numpy as np
import pandas as pd
import seaborn as sns

#check the networkx version (should be 2.2)
print(nx.__version__)

#this is insane
warnings.filterwarnings("ignore",category=cb.MatplotlibDeprecationWarning)

#### Making a basic graph with networkx

In [ ]:
G = nx.Graph() 
G.add_edge(1, 2, weight=4.7 )
G.add_edges_from([(3, 4), (4, 5)], color='red')
G.add_edges_from([(1, 2, {'color': 'blue'}), (2, 3, {'weight': 8})])
G[1][2]['weight'] = 4.7
#G.edges[3, 4]['weight'] = 4.2

nx.draw_networkx(G)

#### Experimenting with Different Graph Generators

In [ ]:
lollipop = nx.lollipop_graph(10,40)
plt.figure(figsize=(10,10))
nx.draw_networkx(lollipop, with_labels=True, font_weight='bold')

In [ ]:
petersen = nx.petersen_graph()
plt.figure(figsize=(10,10))
nx.draw_networkx(petersen, with_labels=True, font_weight='bold')

In [ ]:
tutte = nx.tutte_graph()
plt.figure(figsize=(10,10))
nx.draw_networkx(tutte, with_labels=True, font_weight='bold')

In [ ]:
maze = nx.sedgewick_maze_graph()

plt.figure(figsize=(10,10))
nx.draw_networkx(maze, with_labels=True, font_weight='bold')

In [ ]:
tet = nx.tetrahedral_graph()
plt.figure(figsize=(10,10))
nx.draw_networkx(tet, with_labels=True, font_weight='bold')

#### Degree Distribution

#### Explore Degree Distibution

In [ ]:
#Plot the degree distribution for a random graph

random_graph = nx.gnp_random_graph(1000,0.2)

degrees = list(dict(random_graph.degree()).values())

#look at the range of degrees for the nodes
#print(degrees)

#generate a basic degree histogram
plt.hist(degrees,bins=100)
plt.title('degree histogram for random network')
plt.xlabel('k')
plt.ylabel('P(k)')
plt.show()

#generate a count table
deg_info = np.unique(degrees,return_counts=True)

#plot the log-log plot
plt.scatter(np.log(deg_info[0]),np.log(deg_info[1]))
plt.title('degree distribution for random network')
plt.xlabel('log(k)')
plt.ylabel('log(P(k))')
plt.show()


In [ ]:
#Plot the degree distribution for a scale free graph

scale_free_graph = nx.scale_free_graph(1000,0.15,0.65,0.20)

degrees = list(dict(scale_free_graph.degree()).values())

#look at the range of degrees for the nodes
#print(degrees)

#generate a basic degree histogram
plt.hist(degrees,bins=100)
plt.title('degree histogram for scale-free network')
plt.xlabel('k')
plt.ylabel('P(k)')
plt.show()

#generate a count table
deg_info = np.unique(degrees,return_counts=True)
    
#plot the log-log plot
plt.scatter(np.log(deg_info[0]),np.log(deg_info[1]))
plt.title('degree distribution for scale-free network')
plt.xlabel('log(k)')
plt.ylabel('log(P(k))')
plt.show()

In [ ]:
#Plot the degree distribution for a real biological network
plt.style.use('seaborn')

#load in pair lists for a protein-protein interaction network
alzG = nx.read_edgelist('alz_final.txt')

degrees = list(dict(alzG.degree()).values())

#look at the range of degrees for the nodes
#print(degrees)

#generate a basic degree histogram
plt.hist(degrees,bins=100)
plt.title('degree histogram for Alzheimer\'s Network')
plt.xlabel('k')
plt.ylabel('P(k)')
plt.show()

#generate a count table
deg_info = np.unique(degrees,return_counts=True)

#plot the log-log plot
plt.scatter(np.log(deg_info[0]),np.log(deg_info[1]))
plt.title('degree distribution for Alzheimer\'s network')
plt.xlabel('log(k)')
plt.ylabel('log(P(k))')
plt.show()

plt.figure(figsize=(10,10))
plt.title('Protein-protein Interaction Network for Alzheimer\'s Associated Proteins')
nx.draw_networkx(alzG,
                 with_labels=False,
                 font_weight='bold',
                 node_size=5,
                 edge_color='grey',
                 node_color='blue'
                )
plt.show()

In [ ]:
#perform Girvan-Newman community clustering (makes generator class)
import networkx.algorithms.community.centrality as cn

#this produces an iterator (multiple calls dig deeper into the comm. structure)
gn_clust = cn.girvan_newman(alzG)

#we're going to iterate 15 times
for i in iter(list(range(14))):
    next(gn_clust)
    
# SUGGESTION - Vary the iterations and see what happens.

#final GN iteration
solution= next(gn_clust)

#get the node list
nodes = alzG.nodes()

#create a colour string to colour nodes by cluster
node_cluster={}
cl_number = 0
for cluster in solution:
    for node in nodes:
        if node in cluster:
            node_cluster[node] = cl_number
    cl_number += 1

#create a node attribute to hold the cluster identifier
nx.set_node_attributes(alzG,node_cluster,'cluster')

#get those values back in the correct order for plotting by pulling the by attribute id
node_clusters = nx.get_node_attributes(alzG,'cluster')

#Plot the graph

#user seaborn pallette
sns.set()

plt.figure(figsize=(10,10))

nx.draw_networkx(alzG,
                 with_labels=False,
                 font_weight='bold',
                 node_size=5,
                 edge_color='grey',
                 #list of dict values accesess the INTs
                 node_color=list(node_clusters.values())
                )
plt.show()

In [ ]:
# Gene Expression Correlation Network
# Comparing gene profiles of RNA-seq data between Autistic and Unaffected Individuals

asdVscon_pre = pd.read_csv('AutismVsControl_Pre_Net.tsv',delimiter='\t',names=('gene1','gene2','corr'))

#read in the edges
G2 = nx.from_pandas_edgelist(asdVscon_pre,'gene1','gene2',edge_attr='corr')

nodes = G2.nodes()

#Girvan-Newman Clustering
pre_clust = cn.girvan_newman(G2)

for i in iter(list(range(14))):
    next(pre_clust)

solution = next(pre_clust)

#create a colour string to colour nodes by cluster
node_cluster={}
cl_number = 0
for cluster in solution:
    for node in nodes:
        if node in cluster:
            node_cluster[node] = cl_number
    cl_number += 1

#create a node attribute to hold the cluster identifier
nx.set_node_attributes(G2,node_cluster,'cluster')

#get those values back in the correct order for plotting by pulling the by attribute id
node_clusters = nx.get_node_attributes(G2,'cluster')

#Plot the graph
plt.figure(figsize=(10,10))

nx.draw_networkx(G2,
                 with_labels=False,
                 font_weight='bold',
                 node_size=5,
                 edge_color='grey',
                 #list of dict values accesess the INTs
                 node_color=list(node_clusters.values())
                )

In [ ]:
#Re-plot scaling node_size by betweenness centrality

#betweenness centrality
bc = nx.betweenness_centrality(G2)

#add as a node attribute
nx.set_node_attributes(G2,bc,'betweenness')
#reteive the bcs in order
node_bcs = list(nx.get_node_attributes(G2,'betweenness').values())
#scale them to use as node_size values
node_bcs = [x * 1000 for x in node_bcs]

plt.figure(figsize=(10,10))

nx.draw_networkx(G2,
                 with_labels=False,
                 font_weight='bold',
                 node_size=node_bcs,
                 edge_color='grey',
                 #list of dict values accesess the INTs
                 node_color=list(node_clusters.values())
                )

# SUGGESTION - Which nodes have the highest betweenness centrality?
# SUGGESTION - The node IDs are EntrezID Accessions (GeneIDs), find out what genes these are (you could use BioPython)
# SUGGESTION - Explore some other key network metrics offered by networkx